In [ ]:
import torch
from tensorboard.compat.tensorflow_stub.dtypes import bfloat16

In [ ]:
def main():

In [ ]:
def tensors_memory():

几乎所有东西（参数、梯度、激活值、优化器状态）都存储为浮点数。

# float32
float32 数据类型（也称为 fp32 或单精度）是默认类型。
传统上，在科学计算中，float32 是基准；在某些情况下，你可以使用双精度（float64）。
在深度学习领域，你可以不用那么严谨。
让我们检查一下这些张量的内存使用情况。
内存由（i）值的数量和（ii）每个值的数据类型决定。


In [ ]:
    x = torch.zeros(4, 8)
    assert x.dtype == torch.float32
    assert x.numel() == 4 * 8
    assert x.element_size() == 4
    assert get_memory_usage(x) == 4 * 8 * 4

GPT-3 前馈层中的一个矩阵：

In [ ]:
assert get_memory_usage(torch.empty(12288 * 4, 12288)) == 2304 * 1024 * 1024 # 2.3 GB

# float16
sign(1B) + exponent(5B) + fraction(10B)
float16 数据类型（也称为 fp16 或半精度）减少了内存占用。


In [ ]:
x = torch.zeros(4, 8, dtype=torch.float16)
assert x.element_size() == 2


然而，动态范围（尤其是对于小数字而言）并不理想。

In [ ]:
x = torch.tensor([1e-8], dtype=torch.float16)
assert x == 0

如果在训练时发生这种情况，可能会导致不稳定性。

# bfloat16
Sign(1B) + exponent(8B) + fraction(7B)
谷歌大脑（Google Brain）于 2018 年开发了 bfloat（脑浮点）来解决这一问题。
bfloat16 使用与 float16 相同的内存，但具有与 float32 相同的动态范围！
唯一的问题是分辨率较低，但这对深度学习来说影响较小。


In [ ]:
x = torch.tensor([1e-8], dtype=torch.bfloat16)
assert x != 0

让我们比较一下不同数据类型的动态范围和内存使用情况：

In [ ]:
float32_info = torch.finfo(torch.float32)
print(float32_info)
float16_info = torch.finfo(torch.float16)
print(float16_info)
bfloat16_info = torch.finfo(torch.bfloat16)
print(bfloat16_info)

# fp8
FP8 E4M3: sign(1B) + exponent(4B) + fraction(3B)
FP8 E5M2: sign(1B) + exponent(5B) + fraction(2B)
2022 年，受机器学习工作负载的推动，FP8 实现了标准化。
H100 支持两种 FP8 变体：E4M3（范围 [-448，448]）和 E5M2（[-57344，57344]）。

对训练的影响：
(1)使用 float32 进行训练是可行的，但需要大量内存。
(2)使用 fp8、float16 甚至 bfloat16 进行训练是有风险的，可能会出现不稳定性。
(3)解决方案（稍后）：使用混合精度训练，参见 [mixed_precision_training]()




In [ ]:
def tensors_on_gpus():


默认情况下，张量存储在 CPU 内存中。

In [ ]:
x = torch.zeros(32, 32)
assert x.device == torch.device('cpu')

然而，为了利用 GPU 的大规模并行性，我们需要将它们移至 GPU 内存中。

In [ ]:
我们先看看有没有任何 GPU。

In [ ]:
if not torch.cuda.is_available():
    return

num_gpus = torch.cuda.device_count()
for i in range(num_gpus):
    properties = torch.cuda.get_device_properties(i)

memory_allocated = torch.cuda.memory_allocated()

text("Move the tensor to GPU memory (device 0).")
y = x.to("cuda:0")
assert y.device == torch.device('cuda', 0)

text("Or create a tensor directly on the GPU.")
z = torch.zeros(32, 32, device="cuda:0")

new_memory_allocated = torch.cuda.memory_allocated()
memory_used = new_memory_allocated - memory_allocated
assert memory_used == 2 * (32 * 32 * 4)

In [ ]:
def tensor_operations():


大多数张量是通过对其他张量执行运算而创建的。
每个操作都有一定的内存和计算后果。

In [ ]:
tensor_storage()
tensor_slicing()
tensor_elementwise()
tensor_matmul()

In [ ]:
def tensor_storage():


PyTorch 中的张量是什么？

PyTorch 张量是指向已分配内存的指针

…… 以及描述如何获取张量中任意元素的元数据。


In [ ]:
x = torch.tensor([
    [0, 1, 2, 3],
    [4, 5, 6, 7],
    [8, 9, 10, 11],
    [12, 13, 14, 15]
])

要进入下一行（维度 0），在存储中跳过 4 个元素。

In [ ]:
assert x.stride(0) == 4

要转到下一列（维度 1），需在存储中跳过 1 个元素。

In [ ]:
assert  x.stride(1) == 1

查找元素

In [ ]:
 r, c = 1, 2
index = r * x.stride(0) + c * x.stride(1)  # @inspect index
assert index == 6

In [ ]:
def tensor_slicing():
    x = torch.tensor([[1, 2, 3], [4, 5, 6]])

许多操作只是提供了张量的不同视图。

这不会创建副本，因此一个张量中的突变会影响另一个张量。

获取 0 行

In [ ]:
y = x[0]
assert torch.equal(y, torch.tensor([1, 2, 3]))
assert same_storage(x, y)

获取第1行

In [ ]:
y = x[:, 1]
assert torch.euqal(y, torch.tensor([2, 5]))
assert same_storage(x, y)

view

In [ ]:
y = x.view(3, 2)
assert torch.equal(y, torch.tensor([[1, 2], [3, 4], [5, 6]]))
assert same_storage(x, y)

矩阵转置

In [ ]:
y = x.transpose(1, 0)
assert torch.equal(y, torch.tensor([[1, 4], [2, 5], [3, 6]]))
assert same_storage(x, y)

校验操作x就是操作y


In [ ]:
x[0][0] = 100
assert y[0][0] == 100

请注意，有些视图是不连续的条目，这意味着无法进行更多的视图操作。

In [ ]:
x = torch.tensor([[1., 2, 3], [4, 5, 6]])
y = x.transpose(1, 0)
assert not y.is_contiguous()
try:
    y.view(2, 3)
    assert False
except RuntimeError as e:
    assert "view size is not compatible with input tensor's size and stride" in str(e)

可以先强制一个张量为连续的：

In [ ]:
y = x.transpose(1, 0).contiguous().view(2, 3)
assert not same_storage(x, y)

视图是免费的，复制则会占用（额外的）内存和计算资源。

In [ ]:
def tensor_elementwise():

这些操作会对张量的每个元素执行某种运算
... 并返回一个（新的）形状相同的张量。


In [ ]:
x = torch.tensor([1, 4, 9])
assert torch.equal(x.pow(2), torch.tensor([1, 16, 81]))
assert torch.equal(x.sqrt(), torch.tensor([1, 2, 3]))
assert torch.equal(x.rsqrt(), torch.tensor([1, 1 / 2, 1 / 3]))

assert torch.equal(x + x, torch.tensor([2, 8, 18]))
assert torch.equal(x * 2, torch.tensor([2, 8, 18]))
assert torch.equal(x / 0.5, torch.tensor([2, 8, 18]))

triu 函数取矩阵的上三角部分

In [ ]:
 x = torch.ones(3, 3).triu()  # @inspect x
assert torch.equal(x, torch.tensor([
    [1, 1, 1],
    [0, 1, 1],
    [0, 0, 1]],))

这对于计算因果注意力掩码很有用，其中 M [i, j] 是 i 对 j 的贡献。

In [ ]:
def tensor_matmul():

    最后，深度学习的核心所在：矩阵乘法

In [ ]:
    x = torch.ones(16, 32)
    w = torch.ones(32, 2)
    y = x @ w
    assert y.size() == torch.Size([16, 2])

    通常，我们会对批次中的每个样本和序列中的每个标记执行操作。

In [ ]:
    x = torch.ones(4, 8, 16, 32)
    w = torch.ones(32, 2)
    y = x @ w
    assert y.size() == torch.Size([4, 8, 16, 2])

    在这种情况下，我们遍历 x 的前两个维度的值，并将其与 w 相乘。

In [ ]:
def tensor_einops():
    einops_motivation()

    Einops 是一个用于处理具有命名维度的张量的库.

    它的灵感来源于爱因斯坦求和符号（爱因斯坦，1916 年）。

In [ ]:
    jaxtyping_basics()
    einops_einsum()
    einops_reduce()
    einops_rearrange()

In [ ]:
def einops_motivation():

    传统的 PyTorch 代码：


In [ ]:
    x = torch.ones(2, 2, 3)  # batch, sequence, hidden  @inspect x
    y = torch.ones(2, 2, 3)  # batch, sequence, hidden  @inspect y
    z = x @ y.transpose(-2, -1)  # batch, sequence, sequence  @inspect z

    很容易弄混维度（-2、-1 是什么？）……

In [ ]:
def jaxtyping_basics():

    你是如何跟踪张量维度的？

    原始方法


In [ ]:
    x = torch.ones(2, 2, 1, 3)

    新（jaxtyping）方式

In [ ]:
    x: Float[torch.Tensor, "batch seq heads hidden"] = torch.ones(2, 2, 1, 3)  # @inspect x

    注意：这只是文档说明（无强制力）。

In [ ]:
def einops_einsum():

    爱因斯坦求和（Einsum）是具有良好记账功能的广义矩阵乘法。

    定义两个张量


In [ ]:
     x: Float[torch.Tensor, "batch seq1 hidden"] = torch.ones(2, 3, 4)
     y: Float[torch.Tensor, "batch seq2 hidden"] = torch.ones(2, 3, 4)

    老方法：

In [ ]:
    z = x @ y.transpose(-2, -1)

    新(einops) 方法:

In [ ]:
    z = einsum(x, y, "batch seq1 hidden, batch seq2 hidden -> batch seq1 seq2")

    未在输出中命名的维度会被求和。


    或者可以使用…… 来表示在任意数量的维度上进行广播：

In [ ]:
    z = einsum(x, y, "... seq1 hidden, ... seq2 hidden -> ... seq1 seq2")

In [ ]:
def einops_reduce():

    你可以通过某些运算（例如求和、求均值、求最大值、求最小值）对单个张量进行缩减。

In [ ]:
    x: Float[torch.Tensor, "batch seq hidden"] = torch.ones(2, 3, 4)

    老方法

In [ ]:
    y = x.mean(dim=-1)

    新 (einops) 实现方式:

In [ ]:
    y = reduce(x, "... hidden -> ...", "sum")

In [ ]:
def einops_rearrange():

    有时候，一个维度代表着两个维度
    …… 并且你想要对其中一个进行操作。

In [ ]:
    x: Float[torch.Tensor, "batch seq total_hidden"] = torch.ones(2, 3, 8)

    ... 其中 total_hidden 是 heads * hidden1 的扁平化表示

In [ ]:
    w: Float[torch.Tensor, "hidden1 hidden2"] = torch.ones(4, 4)

    将 total_hidden 拆分为两个维度（头数和 hidden1）：

In [ ]:
    x = rearrange(x, "... (heads hidden1) -> ... heads hidden1", heads=2)

    通过 w 执行转换：

In [ ]:
    x = einsum(x, w, "... hidden1, hidden1 hidden2 -> ... hidden2")

    将头和 hidden2 重新组合在一起：

In [ ]:
    x = rearrange(x, "... heads hidden2 -> ... (heads hidden2)")

In [ ]:
def tensor_operations_flops():

    在完成所有操作后，让我们检查一下它们的计算成本。

    浮点运算（FLOP）是一种基本运算，例如加法（x + y）或乘法（x y）。

    两个极其令人困惑的首字母缩写词（发音相同！）：

    （1）FLOPs：浮点运算（衡量已完成的计算量）

    （2）FLOP/s：每秒浮点运算次数（也写作 FLOPS），用于衡量硬件的速度。

    # 直觉

    训练 GPT-3（2020 年）需要 3.14e23 次浮点运算；

    据推测，训练 GPT-4（2023 年）需要 2e25 次浮点运算

    美国行政命令：任何使用≥1e26 次浮点运算训练的基础模型都必须向政府报告

    A100 的峰值性能为 312 万亿次浮点运算 / 秒

In [ ]:
    assert a100_flop_per_sec == 312e12

    H100 在启用稀疏性时的峰值性能为 1979 万亿次每秒浮点运算，不启用时为 50%

In [ ]:
    assert h100_flop_per_sec == 1979e12 / 2

# 线性模型

    作为动机，假设你有一个线性模型

   + 我们有 n 个点

   + 每个点都是 d 维的

   + 线性模型将每个 d 维向量映射到 k 个输出



In [ ]:
    if  torch.cuda.is_available():
        B = 16384
        D = 32768
        K = 8192
    else:
        B = 1024
        D = 256
        K = 64

    device = get_device()
    x = torch.ones(B, D, device=device)
    w = torch.randn(D, K, device=device)
    y = x @ w

    我们每个（i，j，k）三元组有一次乘法运算（x [i][j] * w [j][k]）和一次加法运算。

In [ ]:
     actual_num_flops = 2 * B * D * K

# 其他操作的浮点运算次数

+ 对一个 m×n 矩阵进行逐元素运算需要 O (mn) 次浮点运算;

+ 两个 m×n 矩阵相加需要 m×n 次浮点运算

一般来说，在深度学习中，对于足够大的矩阵，你所遇到的其他任何运算都不会像矩阵乘法那样耗费资源。

解释：

+ B 是数据点的数量；

+ （D K）是参数的数量；

+ 前向传播的浮点运算次数（FLOPs）为 2×（令牌数量）×（参数数量）

事实证明，这可以推广到 Transformer 模型（作为一级近似

我们的 FLOPs 计算如何转换为实际运行时间（秒）？

我们来计时吧！


In [ ]:
 actual_time = time_matmul(x, w)
 actual_flop_per_sec = actual_num_flops / actual_time

每个 GPU 都有一份规格表，上面标注了峰值性能

+ A100

+ H100

请注意，每秒浮点运算次数（FLOP/s）很大程度上取决于数据类型！

In [ ]:
    promised_flop_per_sec = get_promised_flop_per_sec(device, x.dtype)

# 模型浮点运算次数利用率（MFU）

定义：（实际每秒浮点运算次数）/（承诺的每秒浮点运算次数）

In [ ]:
 mfu = actual_flop_per_sec / promised_flop_per_sec

通常，MFU 大于或等于 0.5 就相当不错了（如果矩阵乘法占主导地位，MFU 会更高）

让我们用 bfloat16 来做：

In [ ]:
    x = x.to(torch.bfloat16)
    w = w.to(torch.bfloat16)
    bf16_actual_time = time_matmul(x, w)
    bf16_actual_flop_per_sec = actual_num_flops / bf16_actual_time
    bf16_promised_flop_per_sec = get_promised_flop_per_sec(device, x.dtype)
    bf16_mfu = bf16_actual_flop_per_sec / bf16_promised_flop_per_sec

注意：与 float32 相比，bfloat16 的实际每秒浮点运算次数（FLOP/s）更高。

这里的 MFU 相当低，可能是因为承诺的 FLOPs 有点过于乐观了。

# 总结

+ 矩阵乘法占主导地位：（2 m n p）次浮点运算

+ 每秒浮点运算次数（FLOP/s）取决于硬件（H100 远优于 A100）和数据类型（bfloat16 远优于 float32）

+ 模型浮点运算利用率（MFU）：（实际每秒浮点运算次数）/（标称每秒浮点运算次数）

In [ ]:
def gradients_basics():

    到目前为止，我们已经构建了张量（它们要么对应参数，要么对应数据），并通过运算（前向传播）传递了这些张量。

    现在，我们要计算梯度（反向传播）

    作为一个简单的例子，让我们来考虑这个简单的线性模型：

    y = 0.5 (x * w - 5)^2

    前向传播：计算损失


In [ ]:
    x = torch.tensor([1., 2, 3])
    w = torch.tensor([1., 1, 1], requires_grad=True)  # Want gradient
    pred_y = x @ w
    loss = 0.5 * (pred_y - 5).pow(2)

    反向传播：计算梯度

In [ ]:
    loss.backward()
    assert loss.grad is None
    assert pred_y.grad is None
    assert x.grad is None
    assert torch.equal(w.grad, torch.tensor([1, 2, 3]))

In [ ]:
def gradients_flops():

    让我们来计算梯度计算的浮点运算次数（FLOPs）


    重新审视我们的线性模型

In [ ]:
    if torch.cuda.is_available():
        B = 16384
        D = 32768
        K = 8192
    else:
        B = 1024
        D = 256
        K = 64

    device = get_device()
    x = torch.ones(B, D, device=device)
    w1 = torch.randn(D, D, device=device, requires_grad=True)
    w2 = torch.randn(D, K, device=device, requires_grad=True)

    Model: x --w1--> h1 --w2--> h2 -> loss

In [ ]:
    h1 = x @ w1
    h2 = h1 @ w2
    loss = h2.pow(2).mean()

    回顾前向 FLOPs 的数量：tensor_operations_flops

    + 计算 x [i][j] 乘以 w1 [j][k]

    + 加上 h1 [i][k]

    + 将 h1 [i][j] 乘以 w2 [j][k]

    +  加上h2[i][k]

In [ ]:
    num_forward_flops = (2 * B * D * D) + (2 * B * D * K)

    运行反向传播需要多少 FLOPs？

In [ ]:
    h1.retain_grad()
    h2.retain_grad()
    loss.backward()

召回模型：x --w1--> h1 --w2--> h2 -> 损失

+ h1.grad = d loss / d h1

+ h2.grad = d loss / d h2

+ w1.grad = d loss / d w1

+ w2.grad = d loss / d w2

关注参数 w2。

应用链式法则。

In [ ]:
num_backward_flops = 0

w2.grad[j,k] = sum_i h1[i,j] * h2.grad[i,k]

In [ ]:
    assert w2.grad.size() == torch.Size([D, K])
    assert h1.size() == torch.Size([B, D])
    assert h2.grad.size() == torch.Size([B, K])

For each (i, j, k), multiply and add.

In [ ]:
     num_backward_flops += 2 * B * D * K

      h1.grad[i,j] = sum_k w2[j,k] * h2.grad[i,k]


In [ ]:
    assert h1.grad.size() == torch.Size([B, D])
    assert w2.size() == torch.Size([D, K])
    assert h2.grad.size() == torch.Size([B, K])

对于每一个（i，j，k），进行乘法和加法运算。

In [ ]:
num_backward_flops += 2 * B * D * K

这只是针对 w2（D*K 参数）的情况

对于 w1（D*D 参数）也可以做到（不过不需要 x.grad）

num_backward_flops += (2 + 2) * B * D * D

In [ ]:
def module_parameters():
    input_dim = 16384
    output_dim = 32

    模型参数在 PyTorch 中存储为 nn.Parameter 对象。

In [ ]:
    w = nn.Parameter(torch.randn(input_dim, output_dim))
    assert isinstance(w, torch.Tensor)
    assert type(w.data) == torch.Tensor

# 参数初始化

让我们看看会发生什么。

In [ ]:
    x = nn.Parameter(torch.randn(input_dim))
    output = x @ w
    assert output.size() == torch.Size([output_dim])

请注意，输出的每个元素都按输入维度的平方根缩放：18.919979095458984。

较大的值可能会导致梯度爆炸，进而使训练不稳定。

我们希望有一种不受 input_dim 影响的初始化方式。

要做到这一点，我们只需按 1/√(input_dim) 进行重新缩放

In [ ]:
 w = nn.Parameter(torch.randn(input_dim, output_dim) / np.sqrt(input_dim))
 output = x @ w

现在输出的每个元素都是常数：-1.5302726030349731。

在常数范围内，这就是 Xavier 初始化。

为了更加安全，我们将正态分布截断到 [-3, 3] 区间，以避免出现任何异常值的可能。



In [ ]:
w = nn.Parameter(nn.init.trunc_normal_(torch.empty(input_dim, output_dim), std=1 / np.sqrt(input_dim), a=-3, b=3))

In [ ]:
def custom_model():

让我们使用 nn.Parameter 构建一个简单的深度线性模型。

In [ ]:
    D = 64  # Dimension
    num_layers = 2
    model = Cruncher(dim=D, num_layers=num_layers)

    param_sizes = [
        (name, param.numel())
        for name, param in model.state_dict().items()
    ]
    assert param_sizes == [
        ("layers.0.weight", D * D),
        ("layers.1.weight", D * D),
        ("final.weight", D),
    ]
    num_parameters = get_num_parameters(model)
    assert num_parameters == (D * D) + (D * D) + D

记得将模型移至 GPU。

In [ ]:
    device = get_device()
    model = model.to(device)

在一些数据上运行该模型。

In [ ]:
    B = 8  # Batch size
    x = torch.randn(B, D, device=device)
    y = model(x)
    assert y.size() == torch.Size([B])

In [ ]:
class Linear(nn.Module):
    """Simple linear layer."""
    def __init__(self, input_dim: int, output_dim: int):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(input_dim, output_dim) / np.sqrt(input_dim))
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return x @ self.weight

In [ ]:
class Cruncher(nn.Module):
    def __init__(self, dim: int, num_layers: int):
        super().__init__()
        self.layers = nn.ModuleList([
            Linear(dim, dim)
            for i in range(num_layers)
        ])
        self.final = Linear(dim, 1)
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # Apply linear layers
        B, D = x.size()
        for layer in self.layers:
            x = layer(x)
        # Apply final head
        x = self.final(x)
        assert x.size() == torch.Size([B, 1])
        # Remove the last dimension
        x = x.squeeze(-1)
        assert x.size() == torch.Size([B])
        return x

In [ ]:
def get_batch(data: np.array, batch_size: int, sequence_length: int, device: str) -> torch.Tensor:


将样本批量大小的随机位置放入数据中。

In [ ]:
    start_indices = torch.randint(len(data) - sequence_length, (batch_size,))
    assert start_indices.size() == torch.Size([batch_size])

对数据进行索引。

In [ ]:
    x = torch.tensor([data[start:start + sequence_length] for start in start_indices])
    assert x.size() == torch.Size([batch_size, sequence_length])

# 固定内存

默认情况下，CPU 张量位于分页内存中。我们可以显式地进行固定。

In [ ]:
    if torch.cuda.is_available():
        x = x.pin_memory()

这使我们能够将 x 从 CPU 异步复制到 GPU。

In [ ]:
    x = x.to(device, non_blocking=True)

这使我们能够并行执行两项操作（此处未执行）：

+ 将下一批数据读取到中央处理器（CPU）中

+ 在 GPU 上处理 x。

[article](https://developer.nvidia.com/blog/how-optimize-data-transfers-cuda-cc/)

[article](https://gist.github.com/ZijiaLewisLu/eabdca955110833c0ce984d34eb7ff39?permalink_comment_id=3417135)

In [ ]:
  return x

In [ ]:
def note_about_randomness():

随机性出现在许多地方：参数初始化、 dropout、数据排序等等。

为了保证可复现性，我们建议在每次使用随机性时都传入一个不同的随机种子。

确定性在调试时特别有用，这样你就能找到程序缺陷了。

有三个地方可以设置随机种子，为了安全起见，你应该一次性全部设置好。

In [ ]:
    # Torch
    seed = 0
    torch.manual_seed(seed)
    # NumPy
    import numpy as np
    np.random.seed(seed)
    # Python
    import random
    random.seed(seed)

In [ ]:
def data_loading():

在语言建模中，数据是一系列整数（由分词器输出）。

将它们序列化为 numpy 数组很方便（这由分词器完成）。

In [ ]:
    orig_data = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], dtype=np.int32)
    orig_data.tofile("data.npy")

你可以将它们重新加载为 numpy 数组。

不想一次性将全部数据加载到内存中（LLaMA 数据为 2.8TB）

使用内存映射（memmap）来延迟加载，只将被访问的部分加载到内存中。

In [ ]:
    data = np.memmap("data.npy", dtype=np.int32)
    assert np.array_equal(data, orig_data)

数据加载器会生成一批用于训练的序列。

In [ ]:
    B = 2  # Batch size
    L = 4  # Length of sequence
    x = get_batch(data, batch_size=B, sequence_length=L, device=get_device())
    assert x.size() == torch.Size([B, L])

In [ ]:
class SGD(torch.optim.Optimizer):
    def __init__(self, params: Iterable[nn.Parameter], lr: float = 0.01):
        super(SGD, self).__init__(params, dict(lr=lr))
    def step(self):
        for group in self.param_groups:
            lr = group["lr"]
            for p in group["params"]:
                grad = p.grad.data
                p.data -= lr * grad

In [ ]:
class AdaGrad(torch.optim.Optimizer):
    def __init__(self, params: Iterable[nn.Parameter], lr: float = 0.01):
        super(AdaGrad, self).__init__(params, dict(lr=lr))
    def step(self):
        for group in self.param_groups:
            lr = group["lr"]
            for p in group["params"]:
                # Optimizer state
                state = self.state[p]
                grad = p.grad.data
                # Get squared gradients g2 = sum_{i<t} g_i^2
                g2 = state.get("g2", torch.zeros_like(grad))
                # Update optimizer state
                g2 += torch.square(grad)
                state["g2"] = g2
                # Update parameters
                p.data -= lr * grad / torch.sqrt(g2 + 1e-5)

In [ ]:
def optimizer():

回想一下我们的深度线性模型。

In [ ]:
    B = 2
    D = 4
    num_layers = 2
    model = Cruncher(dim=D, num_layers=num_layers).to(get_device())

让我们来定义 AdaGrad 优化器

+ 动量 = 随机梯度下降 + 梯度的指数平均

+ AdaGrad = 随机梯度下降 + 按梯度平方进行平均

+ RMSProp = AdaGrad + 梯度平方的指数平均

+ Adam = 均方根传播（RMSProp） + 动量（momentum）

[AdaGrad](https://www.jmlr.org/papers/volume12/duchi11a/duchi11a.pdf)

In [ ]:
    optimizer = AdaGrad(model.parameters(), lr=0.01)
    state = model.state_dict()  # @inspect state

计算梯度

In [ ]:
    x = torch.randn(B, D, device=get_device())
    y = torch.tensor([4., 5.], device=get_device())
    pred_y = model(x)
    loss = F.mse_loss(input=pred_y, target=y)
    loss.backward()

迈出一步

In [ ]:
    optimizer.step()
    state = model.state_dict()

释放内存（可选）

In [ ]:
optimizer.zero_grad(set_to_none=True)

# 内存

In [ ]:
    # Parameters
    num_parameters = (D * D * num_layers) + D  # @inspect num_parameters
    assert num_parameters == get_num_parameters(model)

In [ ]:
    # Activations
    num_activations = B * D * num_layers  # @inspect num_activations

In [ ]:
    # Gradients
    num_gradients = num_parameters  # @inspect num_gradients

In [ ]:
    # Optimizer states
    num_optimizer_states = num_parameters  # @inspect num_optimizer_states

In [ ]:
    # Putting it all together, assuming float32
    total_memory = 4 * (num_parameters + num_activations + num_gradients + num_optimizer_states)

# 计算

In [ ]:
 flops = 6 * B * num_parameters

# Transformers

Transformer 的会计处理更为复杂，但原理是一样的。

作业 1 会要求你做那件事。

描述 Transformer 训练内存使用情况的博客文章 [文章](https://erees.dev/transformer-memory/)

描述 Transformer 的 FLOPs 的博客文章： [文章](https://www.adamcasson.com/posts/transformer-flops)

In [ ]:
def train_loop():

从权重为（0，1，2，……，D-1）的线性函数生成数据。

In [ ]:
    D = 16
    true_w = torch.arange(D, dtype=torch.float32, device=get_device())
    def get_batch(B: int) -> tuple[torch.Tensor, torch.Tensor]:
        x = torch.randn(B, D).to(get_device())
        true_y = x @ true_w
        return (x, true_y)

我们来进行一次基本的运行

In [ ]:
    train("simple", get_batch, D=D, num_layers=0, B=4, num_train_steps=10, lr=0.01)

进行一些超参数调优

In [ ]:
    train("simple", get_batch, D=D, num_layers=0, B=4, num_train_steps=10, lr=0.1)

In [ ]:
def train(name: str, get_batch,
          D: int, num_layers: int,
          B: int, num_train_steps: int, lr: float):
    model = Cruncher(dim=D, num_layers=0).to(get_device())
    optimizer = SGD(model.parameters(), lr=0.01)
    for t in range(num_train_steps):
        # Get data
        x, y = get_batch(B=B)
        # Forward (compute loss)
        pred_y = model(x)
        loss = F.mse_loss(pred_y, y)
        # Backward (compute gradients)
        loss.backward()
        # Update parameters
        optimizer.step()
        optimizer.zero_grad(set_to_none=True)

In [ ]:
def checkpointing():

训练语言模型需要很长时间，而且肯定会崩溃。

你不会想失去所有的进展。

在训练过程中，定期将模型和优化器的状态保存到磁盘是很有用的。

In [ ]:
    model = Cruncher(dim=64, num_layers=3).to(get_device())
    optimizer = AdaGrad(model.parameters(), lr=0.01)

保存检查点：

In [ ]:
     checkpoint = {
        "model": model.state_dict(),
        "optimizer": optimizer.state_dict(),
    }
    torch.save(checkpoint, "model_checkpoint.pt")

加载检查点：

In [ ]:
    loaded_checkpoint = torch.load("model_checkpoint.pt")

In [ ]:
def mixed_precision_training():

数据类型（float32、bfloat16、fp8）的选择各有取舍。

+ 更高的精度：更准确 / 稳定，需要更多内存，需要更多计算资源

+ 较低精度：精度 / 稳定性较差，内存占用更少，计算量更少

我们怎样才能两全其美呢？

解决方案：默认使用 float32，但在可能的情况下使用 {bfloat16, fp8}。

一个具体的计划：

+ 在前向传播（激活）中使用 {bfloat16, fp8}。

+ 其余部分（参数、梯度）使用 float32。

+ 混合精度训练[Micikevicius+ 2017]([Micikevicius+ 2017])

PyTorch 有一个自动混合精度（AMP）库。
https://pytorch.org/docs/stable/amp.html

https://docs.nvidia.com/deeplearning/performance/mixed-precision-training/

NVIDIA 的 Transformer 引擎支持线性层使用 FP8

在整个训练过程中普遍使用 FP8 [Peng+ 2023](https://arxiv.org/pdf/2310.18313.pdf)

In [ ]:
def get_memory_usage(x: torch.Tensor):
    return x.numel() * x.element_size()
def get_promised_flop_per_sec(device: str, dtype: torch.dtype) -> float:
    """Return the peak FLOP/s for `device` operating on `dtype`."""
    if not torch.cuda.is_available():

没有可用的 CUDA 设备，因此无法获取每秒浮点运算次数（FLOP/s）。


In [ ]:
         return 1
    properties = torch.cuda.get_device_properties(device)

    if "A100" in properties.name:
        # https://www.nvidia.com/content/dam/en-zz/Solutions/Data-Center/a100/pdf/nvidia-a100-datasheet-us-nvidia-1758950-r4-web.pdf")
        if dtype == torch.float32:
            return 19.5e12
        if dtype in (torch.bfloat16, torch.float16):
            return 312e12
        raise ValueError(f"Unknown dtype: {dtype}")
    if "H100" in properties.name:
        # https://resources.nvidia.com/en-us-tensor-core/nvidia-tensor-core-gpu-datasheet")
        if dtype == torch.float32:
            return 67.5e12
        if dtype in (torch.bfloat16, torch.float16):
            return 1979e12 / 2  # 1979 is for sparse, dense is half of that
        raise ValueError(f"Unknown dtype: {dtype}")
    raise ValueError(f"Unknown device: {device}")

In [ ]:
def same_storage(x: torch.Tensor, y: torch.Tensor):
    return x.untyped_storage().data_ptr() == y.untyped_storage().data_ptr()

In [ ]:
def time_matmul(a: torch.Tensor, b: torch.Tensor) -> float:
    """Return the number of seconds required to perform `a @ b`."""
    # Wait until previous CUDA threads are done
    if torch.cuda.is_available():
        torch.cuda.synchronize()
    def run():
        # Perform the operation
        a @ b
        # Wait until CUDA threads are done
        if torch.cuda.is_available():
            torch.cuda.synchronize()
    # Time the operation `num_trials` times
    num_trials = 5
    total_time = timeit.timeit(run, number=num_trials)
    return total_time / num_trials

In [ ]:
def get_num_parameters(model: nn.Module) -> int:
    return sum(param.numel() for param in model.parameters())

In [ ]:
def get_device(index: int = 0) -> torch.device:
    """Try to use the GPU if possible, otherwise, use CPU."""
    if torch.cuda.is_available():
        return torch.device(f"cuda:{index}")
    else:
        return torch.device("cpu")

In [ ]:
if __name__ == "__main__":
    main()